# Long-Short Term Memory Stock Predictor

2024, Niko Bagaric TF practice project

## Imports & Functions

In [ ]:
!pip install alpha_vantage

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from alpha_vantage.timeseries import TimeSeries

In [ ]:
def getStockData(ticker_symbol, ts):
  data, metadata = ts.get_daily(symbol=ticker_symbol, outputsize='full')

  data['SMA_50'] = data['4. close'].rolling(window=50).mean()
  data['SMA_200'] = data['4. close'].rolling(window=200).mean()

  data.dropna(inplace=True)

  features = data[['4. close', 'SMA_50', 'SMA_200']]

  scaler = MinMaxScaler(feature_range=(0, 1))
  scaled_features = scaler.fit_transform(features)

  X, y = [], []
  for i in range(len(scaled_features) - 60):
    X.append(scaled_features[i:i + 60])
    y.append(scaled_features[i + 60, 0])

  X, y = np.array(X), np.array(y)

  return X, y

def makeEvaluationPlot(X1, y1):
  # Evaluate the model
  loss = model.evaluate(X1, y1)

  # Make predictions
  predictions = model.predict(X1)
  predictions = scaler.inverse_transform(np.concatenate((predictions, X1[:, 0, 1:]), axis=1))[:, 0]
  y1 = scaler.inverse_transform(np.concatenate((y1.reshape(-1, 1), X1[:, 0, 1:]), axis=1))[:, 0]

  # Plot the results
  plt.figure(figsize=(14, 5))
  plt.plot(y1, color='blue', label='Actual Stock Price')
  plt.plot(predictions, color='red', label='Predicted Stock Price')
  plt.title('Stock Price Prediction using LSTM')
  plt.xlabel('Time')
  plt.ylabel('Stock Price')
  plt.legend()
  plt.show()

## Data Grabbing

The data is grabbed from [here](https://www.alphavantage.co/)

In [ ]:
# Define API key and TS
api_key = 'YOUR API KEY HERE'
ts = TimeSeries(key=api_key, output_format='pandas')

In [ ]:
# Define the ticker symbol (apple)
ticker_symbol = 'AAPL'

In [ ]:
X, y = getStockData(ticker_symbol=ticker_symbol,
                    ts=ts)

In [ ]:
# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

## Model A-01

### Training

In [ ]:
# Define the LSTM model
model = Sequential([
    LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(units=50, return_sequences=True),
    Dropout(0.2),
    LSTM(units=50),
    Dropout(0.2),
    Dense(units=1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
makeEvaluationPlot(X1=X_test,
                   y1=y_test)

### Testing on different dataset (different stock)

Testing on different stock

In [ ]:
ticker_symbol_nvidia = 'NVDA'

X1, y1 = getStockData(ticker_symbol=ticker_symbol_nvidia,
                      ts=ts)

In [ ]:
# Let's evaluate on a big set

makeEvaluationPlot(X1=X1,
                   y1=y1)

In [ ]:
ticker_symbol_msft = 'MSFT'

X2, y2 = getStockData(ticker_symbol=ticker_symbol_msft,
                      ts=ts)

In [ ]:
pd.read_csv('')